# EMPLOYEE TURNOVER USING SURVIVAL ANALYSIS

# Importing Libraries

In [1]:
pip install sklearn

In [6]:
!pip install pandas-profiling


  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27091 sha256=704a41c58e0ad5025f12e0c04abc0b0866f5822b7d3eeaa7bc8f39240afa2e8a
  Stored in directory: c:\users\akash sharma\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295204 sha256=5079f3cc0c1dffe683989c8b547081662becd7d3c6f6fc8449c28d1950f059a2
  Stored in directory: c:\users\akash sharma\appdata\local\pip\cache\wheels\48\a1\7f\096c1269d6bf78d4768180602579b35a1e8cb1250bb4b40c74
Successfully built htmlmin imagehash
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1


#Before installing pandas profiling module, you should have install Microsoft visual studio 2019 in your desktop > Modify > Workloads > Desktop development with C++ then download and install all the files that it will suggest ,otherwise you will be getting an error.
Use this link : https://visualstudio.microsoft.com/downloads/ (Download the community version)

In [3]:
pip install seaborn


Note: you may need to restart the kernel to use updated packages.


'C:\Users\Akash' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
pip install lifelines

Note: you may need to restart the kernel to use updated packages.


'C:\Users\Akash' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import statistics

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')



ModuleNotFoundError: No module named 'pandas_profiling'

# Importing the data file

In [ ]:
df=pd.read_excel("turnover-Data set Mentee 1.xlsx")
df.head(5)

In [ ]:
df['Joining date']=df['Joining date'].astype(str)

In [ ]:
df[['Unnamed: 17','Unnamed: 18','Unnamed: 19']]=df['Joining date'].str.split('-',expand=True)

In [ ]:
df.rename(columns={'Unnamed: 17' : 'Year','Unnamed: 18' : 'Month','Unnamed: 19' : 'Date'},inplace=True)

In [ ]:
df.dtypes

In [ ]:
df['Year']=df['Year'].astype(int)
df['Month']=df['Month'].astype(int)
df['Date']=df['Date'].astype(int)

In [ ]:
df.dtypes

# Data Description

- stag : The duration of stay (in weeks) of an employee in a job.<br>

- event : 0 - The employee is not terminated from the job.<br>
     1 - The employee is terminated from the job.<br>
          
- gender : The gender of the employee.<br>

- age : The age of the employee.It varies from 18yrs - 58yrs.<br>

- industry : The name of the industry in which the employee works.<br>
Retail           - 289<br>
manufacture      - 145<br>
IT               - 122<br>
Banks            - 114<br>
etc              -  94<br>
Consult          -  74<br>
State            -  55<br>
Building         -  41<br>
transport        -  38<br>
PowerGeneration  -  38<br>
Telecom          -  36<br>
Mining           -  24<br>
Pharma           -  20<br>
Agriculture      -  15<br>
RealEstate       -  13<br>
HoReCa          -  11<br>
    
- profession : The name of the profession that the employee does<br>
HR                  - 757<br>
IT                  -  74<br>
Sales               -  66<br>
etc                 -  37<br>
Marketing           -  31<br>
BusinessDevelopment -  27<br>
Consult             -  25<br>
Commercial          -  23<br>
manage              -  22<br>
Finanñe             -  17<br>
Engineer            -  15<br>
Teaching            -  12<br>
Accounting          -  10<br>
Law                 -   7<br>
PR                  -   6<br>

- traffic : Different sources from where the employees get their job.<br>
youjs        - 318<br>
empjs        - 248<br>
rabrecNErab  - 211<br>
friends      - 118<br>
referal      -  95<br>
KA           -  67<br>
recNErab     -  39<br>
advert       -  33<br>

- coach	: Whether the employee has a coach during/before their job period.<br>
no        - 683<br>
my head   - 314<br>
yes       - 132<br>

- head_gender : Gender of the head manager provided to each employee.<br>
m   - 584<br>
f   - 545<br>

- greywage : Type of wages i.e white(without any black money) and grey(with black money)<br>
white   - 1002<br>
grey    -  127<br>

- way : The mode of transportation of employee.<br>
bus   -  681<br>
car   -  331<br>
foot  -  117<br>

- extraversion : The extraversion rating of employee out of 10. <br>

- independ : The independ rating of employee out of 10.<br>

- selfcontrol : The selfcontrol rating of employee out of 10.<br>

- anxiety : The anxiety rating of employee out of 10.<br>

- novator : The novator rating of employee out of 10.<br>

- Joining date  : The date of joining of employees.The data is of 2 years from 17/1/2012 to 17/1/2014.<br>

- Year : The year of joining.

- Month : The month of joining.

- Date : The date of joining.

- Variable type  :<br> 
 Numeric : stag, age, extraversion, independ,
  selfcontrol, anxiety, novator, Year, Month, Date (10)<br>
  
 Categorical :  event, gender, industry, profession, traffic,
  coach, head_gender, greywage, way (9)<br> 
  
 DateTime : Joining date(1)
   
- Target Variable : stag  

In [ ]:
df.shape

# Data Exploration

In [ ]:
df.event.value_counts()

###  There are 571 employees terminated and 558 employees stayed in the job. We have 571 non-censored data and 558 censored data. 

In [ ]:
df.event.value_counts(normalize=True)

### 49%  of the employee stayed in the job and 51%  of the employee terminated from the job.

In [ ]:
df.groupby('event').mean()

In [ ]:
dt=df.groupby(by='event')
dr=dt.get_group(1)
dr['age'].value_counts()

In [ ]:
#pandas_profiling.ProfileReport(df)

### Joining Date is not useful as we already have the information in other columns.So we will remove it.
###  There is no missing and duplicate value in our data.

In [ ]:
df.drop(['extraversion','novator','profession','Joining date'],axis=1,inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
num_bins = 14
df.hist(bins=num_bins, figsize=(20,15))
plt.show()

# Data Visualization

## Gender

In [ ]:
df['gender'].value_counts()

In [ ]:
df[['gender' , 'event']].groupby('gender').mean().sort_values(by='event')

### The % of female employee terminated  is slightly more than males.

In [ ]:
pd.crosstab(df.gender , df.event, margins=True , normalize=True)

In [ ]:
pd.crosstab(df.gender,df.event).plot(kind='bar',color=('green','red'))
plt.xticks(rotation='horizontal')
plt.grid()
plt.title('Gender wise Turnover frequency')
plt.xlabel('Gender')
plt.ylabel('Frequency of turnover')


In [ ]:
fig_dims=(5,3)
fig,ax=plt.subplots(figsize=fig_dims)
fig,ax=plt.xticks(rotation='horizontal')
fig,ax=plt.ylim(0,0.7)
sns.barplot(x='gender',y='event',data=df,ci=None)

ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=12,color='black',ha='center',va='bottom')
plt.savefig('gender.png',dpi=300)    

### More female is terminated from the job than male.

## Age

In [ ]:
dr=df.groupby('event')
dt=dr.get_group(1)
dt['gender'].value_counts()
#de=dt.groupby('gender')
#t=de.get_group('m')
#w=t['age'].value_counts()
#dt['independ'].value_counts()

In [ ]:
g=sns.catplot( y='age',x='event' , data=df  , kind='violin' , palette=['green','red'],height=4,legend=True)
(g.set_axis_labels("", "Age")
  .set_xticklabels(["Not-terminated" , "terminated"]))

plt.grid()
plt.savefig('age.png',dpi=300)

## Industry

In [ ]:
pd.crosstab(df.industry , df.event,margins=True ,normalize=True)

In [ ]:
df[['industry' , 'event']].groupby('industry').mean().sort_values(by='event')

### % of employee in different industry who terminated from their job.

In [ ]:
fig_dim=(7,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='vertical')
fig,ax=plt.ylim(0,0.9)
sns.barplot(x='industry',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('industry.png',dpi=300)
    

In [ ]:
pd.crosstab(df.industry , df.event ).plot(kind='bar',color=('green','red'),figsize=(10,5))
plt.xticks(rotation='vertical')
plt.grid()
plt.title('Industry wise Turnover frequency')
plt.xlabel('Industry')
plt.ylabel('Frequency of turnover')


### In 'Building' most of the employees is terminated.

### Traffic

In [ ]:
pd.crosstab(df.traffic,df.event,margins=True)

In [ ]:
df[['traffic' , 'event']].groupby('traffic').mean().sort_values(by='event')

In [ ]:
fig_dim=(5,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='vertical')
fig,ax=plt.ylim(0,0.9)
sns.barplot(x='traffic',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('traffic.png',dpi=300)
    

In [ ]:
pd.crosstab(df.traffic,df.event).plot(kind='bar',color=('green','red',),figsize=(10,5))
plt.xticks(rotation='vertical')
plt.grid()
plt.title('Traffic wise Turnover frequency')
plt.xlabel('Traffic')
plt.ylabel('Frequency of turnover')

### Employee who knows about the job through referal are more likely to terminate.

## Coach

In [ ]:
pd.crosstab(df.coach , df.event , margins=True )

In [ ]:
df[['coach','event']].groupby('coach').mean().sort_values(by='event')

In [ ]:
pd.crosstab(df.coach , df.event ).plot(kind='bar',color=('g','r'),figsize=(10,5))
plt.xticks(rotation='horizontal')
plt.grid()
plt.title('Coach wise Turnover frequency')
plt.xlabel('Coach')
plt.ylabel('Frequency of turnover')


In [ ]:
fig_dim=(4,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='horizontal')
fig,ax=plt.ylim(0,0.7)
sns.barplot(x='coach',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('coach.png',dpi=300)
    

### Employee having a coach is more likely to terminate.

## head_gender

In [ ]:
df[['head_gender','event']].groupby('head_gender').mean().sort_values(by='event')

In [ ]:
pd.crosstab(df.head_gender , df.event , margins=True )

In [ ]:
pd.crosstab(df.head_gender , df.event ).plot(kind='bar',color=('g','r'),figsize=(5,3))
plt.xticks(rotation='horizontal')
plt.grid()
plt.title('head_gender wise Turnover frequency')
plt.xlabel('head_gender')
plt.ylabel('Frequency of turnover')

In [ ]:
fig_dim=(3,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='horizontal')
fig,ax=plt.ylim(0,0.7)
sns.barplot(x='head_gender',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('head_gender.png',dpi=300)
    

### Employee with male head are slightly more likely to terminate from their job.

## Greywage

In [ ]:
df[['greywage','event']].groupby('greywage').mean().sort_values(by='event')

In [ ]:
pd.crosstab(df.greywage , df.event ).plot(kind='bar',color=('g','r'),figsize=(7,5))
plt.xticks(rotation='horizontal')
plt.grid()
plt.title('greywage wise Turnover frequency')
plt.xlabel('greywage')
plt.ylabel('Frequency of turnover')

In [ ]:
fig_dim=(3,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='horizontal')
fig,ax=plt.ylim(0,0.7)
sns.barplot(x='greywage',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('greywage.png',dpi=300)

### Employees with grey wages are more likely to terminate from their job.

## Way

In [ ]:
pd.crosstab(df.way , df.event,margins=True )

In [ ]:
pd.crosstab(df.way , df.event).plot(kind='bar',color=('g','r'),figsize=(10,5))
plt.xticks(rotation='horizontal')
plt.grid()
plt.title('Way wise Turnover frequency')
plt.xlabel('Way')
plt.ylabel('Frequency of turnover')

In [ ]:
df[['way' , 'event']].groupby('way').mean().sort_values(by='event')

In [ ]:
fig_dim=(5,3)
fig,ax=plt.subplots(figsize=fig_dim)
fig,ax=plt.xticks(rotation='horizontal')
fig,ax=plt.ylim(0,0.6)
sns.barplot(x='way',y='event',data=df,ci=None)
ax=plt.gca()
for p in ax.patches:
    ax.text(p.get_x()+p.get_width()/2,p.get_height(),'%.2f' % float(p.get_height()),fontsize=8,color='Black',ha='center',va='bottom')
plt.savefig('way.png')    


### 53% employee whose mode of transportation is car is terminated.

## independ

In [ ]:
g=sns.catplot( y='independ',x='event' , data=df  , kind='violin' , palette=['green','red'] , height=4 , legend=True)
(g.set_axis_labels("", "Independent")
  .set_xticklabels(["Not-terminated" , "terminated"]))

plt.grid()
plt.savefig('independ.png',dpi=300)

####  Most of the employee who terminate from their job has a rating in the range 4.8 to 5.5

## Selfcontrol

In [ ]:
df['selfcontrol'].value_counts()

In [ ]:
df[['selfcontrol' , 'event']].groupby('selfcontrol').mean().sort_values(by='event')

In [ ]:
g=sns.catplot( y='selfcontrol',x='event' , data=df  , kind='violin' , palette=['green','red'] , height=4 , legend=True)
(g.set_axis_labels("", "Selfcontrol")
  .set_xticklabels(["Not-terminated" , "terminated"]))

plt.grid()
plt.savefig('selfcontrol.png',dpi=300)

### Most of the employees with rating  6 has terminated from their job.

## Anxiety

In [ ]:
g=sns.catplot( y='anxiety',x='event' , data=df  , kind='violin' , palette=['green','red'] , height=4 , legend=True)
(g.set_axis_labels("", "anxiety")
  .set_xticklabels(["Not-terminated" , "terminated"]))

plt.grid()
plt.savefig('anxiety.png',dpi=300)

### Employee with anxiety rating around 6 are more likely to terminate from their job.

### Kaplan-meier curve

In [ ]:
T = df['stag']
E = df['event']
km = KaplanMeierFitter() 
plt.figure(figsize=(10,7))
km.fit(T, E,label='Kaplan Meier Estimate')
km.plot(ci_show=True) ## ci_show is meant for Confidence interval
plt.grid()
plt.show()
plt.savefig('km0.jpg',dpi=300)

In [ ]:
df['industry']=df['industry'].replace({'PowerGeneration':'Pharma','Telecom':'Pharma','Agriculture':'Pharma','RealEstate':'Pharma','etc':'Pharma',' HoReCa':'Pharma',
       'Mining':'Pharma', 'transport':'Pharma'})

df['industry']=np.where(df['industry'] =='Pharma','Others', df['industry'])
df['industry'].value_counts()

In [ ]:
df.columns

In [ ]:
def map_all(frame):
    # Map Sex
    frame['gender'] = frame.gender.map({'m': 0 ,  'f': 1}).astype(int) 
    # Map profession
    #frame['profession'] = frame.profession.map({'HR' : 0 , 'IT': 1 , 'Consult':2 , 'etc':3 , 'Marketing': 4 ,'BusinessDevelopment' :5 , 'Others': 6}).astype(int) 
    # Map industry
    frame['industry'] = frame.industry.map({'Building' : 0 , 'Consult': 1 , 'Banks':2 , 'Retail':3 , 'manufacture': 4 ,'State' :5 , 'IT':6,'Others': 7}).astype(int)  
    # Map way
    frame['way'] = frame.way.map({'bus' : 0 , 'car': 1 , 'foot':2}).astype(int)   
    # Map traffic
    frame['traffic'] = frame.traffic.map({'youjs' : 0 , 'empjs': 1 , 'rabrecNErab':2 , 'friends':3 , 'referal': 4 ,'KA' :5 , 'recNErab':6, 'advert':7}).astype(int)  
    # Map greywage
    frame['greywage'] = frame.greywage.map({'white' : 0 , 'grey': 1 }).astype(int) 
    # Map coach
    frame['coach'] = frame.coach.map({'no' : 0 , 'yes': 1 , 'my head':2 }).astype(int) 
    # Map head_gender
    frame['head_gender'] = frame.head_gender.map({'m' : 0 , 'f': 1 }).astype(int)  
  

In [ ]:
map_all(df)

In [ ]:
T = df['stag']
E = df['event']

km_2 = KaplanMeierFitter()

groups = df['gender']   
i1 = (groups == 0)   
i2 = (groups == 1)  

plt.figure(figsize=(7,5))
ax=plt.subplot()

## fit the model for 1st group
km_2.fit(T[i1], E[i1], label='male')
km_2.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_2.fit(T[i2], E[i2], label='female')
km_2.plot(ax=ax,ci_show=False)
plt.grid()
plt.show()
plt.savefig('km1.png',dpi=300)



In [ ]:
km_3 = KaplanMeierFitter()

groups=df['age']
g1=(groups<=26)  
g2=(groups>26) & (groups<=30)
g3=(groups>30) & (groups<=36)
g4=(groups>36)
plt.figure(figsize=(7,5))
ax=plt.subplot()

## fit the model for 1st group
km_3.fit(T[g1], E[g1], label='below age 26')
km_3.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_3.fit(T[g2], E[g2], label='age btw 26-30')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g3], E[g3], label='age btw 30-36 ')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g4], E[g4], label='above age 36')
km_3.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km2.png',dpi=300)

In [ ]:
km_4 = KaplanMeierFitter()

groups=df['industry']
g1=(groups==0)  
g2=(groups==1)
g3=(groups==2)
g4=(groups==3)
g5=(groups==4)
g6=(groups==5)
g7=(groups==6)
g8=(groups==7)

plt.figure(figsize=(7,5))
ax=plt.subplot()

## fit the model for 1st group
km_4.fit(T[g1], E[g1], label='Building')
km_4.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_4.fit(T[g2], E[g2], label='Consult')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g3], E[g3], label='Banks')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g4], E[g4], label='Retail')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g5], E[g5], label='manufacture')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g6], E[g6], label='State')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g7], E[g7], label='IT')
km_4.plot(ax=ax,ci_show=False)

km_4.fit(T[g8], E[g8], label='Others')
km_4.plot(ax=ax,ci_show=False)

plt.grid()
plt.show()
plt.savefig('km4.png',dpi=300)

In [ ]:
km_6 = KaplanMeierFitter()

groups=df['coach']
g1=(groups==0)  
g2=(groups==1)
g3=(groups==2)


plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_6.fit(T[g1], E[g1], label='no')
km_6.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_6.fit(T[g2], E[g2], label='yes')
km_6.plot(ax=ax,ci_show=False)

km_6.fit(T[g3], E[g3], label='my_head')
km_6.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km5.png',dpi=300)

In [ ]:
km_6 = KaplanMeierFitter()

groups=df['head_gender']
g1=(groups==0)  
g2=(groups==1)

plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_6.fit(T[g1], E[g1], label='m')
km_6.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_6.fit(T[g2], E[g2], label='f')
km_6.plot(ax=ax,ci_show=False)

plt.grid()
plt.show()
plt.savefig('km6.png',dpi=300)

In [ ]:
km_6 = KaplanMeierFitter()

groups=df['way']
g1=(groups==0)  
g2=(groups==1)
g3=(groups==2)


plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_6.fit(T[g1], E[g1], label='bus')
km_6.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_6.fit(T[g2], E[g2], label='car')
km_6.plot(ax=ax,ci_show=False)

km_6.fit(T[g3], E[g3], label='foot')
km_6.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km7.png',dpi=300)

In [ ]:
km_7 = KaplanMeierFitter()

groups=df['greywage']
g1=(groups==0)  
g2=(groups==1)

plt.figure(figsize=(7,5))
ax=plt.subplot()

## fit the model for 1st group
km_7.fit(T[g1], E[g1], label='white')
km_7.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_7.fit(T[g2], E[g2], label='grey')
km_7.plot(ax=ax,ci_show=False)



plt.grid()
plt.show()
plt.savefig('km8.png',dpi=300)

In [ ]:
km_8 = KaplanMeierFitter()

groups=df['traffic']
g1=(groups==0)  
g2=(groups==1)
g3=(groups==2)
g4=(groups==3)
g5=(groups==4)
g6=(groups==5)
g7=(groups==6)
g8=(groups==7)


plt.figure(figsize=(7,5))
ax=plt.subplot()

## fit the model for 1st group
km_8.fit(T[g1], E[g1], label='youjs')
km_8.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_8.fit(T[g2], E[g2], label='empjs')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g3], E[g3], label='rabrecNErab')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g4], E[g4], label='friends')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g5], E[g5], label='referal')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g6], E[g6], label='KA')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g7], E[g7], label='recNErab')
km_8.plot(ax=ax,ci_show=False)

km_8.fit(T[g8], E[g8], label='advert')
km_8.plot(ax=ax,ci_show=False)
plt.grid()
plt.show()
plt.savefig('km9.png',dpi=300)

In [ ]:
km_3 = KaplanMeierFitter()

groups=df['independ']
g1=(groups<=4.1)  
g2=(groups>4.1) & (groups<=5.5)
g3=(groups>5.5) & (groups<=6.9)
g4=(groups>6.9)
plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_3.fit(T[g1], E[g1], label='below 4.1')
km_3.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_3.fit(T[g2], E[g2], label='btw 4.1-5.5')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g3], E[g3], label='btw 5.5-6.9')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g4], E[g4], label='above 6.9')
km_3.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km10.png',dpi=300)

In [ ]:
km_3 = KaplanMeierFitter()

groups=df['selfcontrol']
g1=(groups<=4.1)  
g2=(groups>4.1) & (groups<=5.7)
g3=(groups>5.7) & (groups<=7.2)
g4=(groups>7.2)
plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_3.fit(T[g1], E[g1], label='below 4.1')
km_3.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_3.fit(T[g2], E[g2], label='btw 4.1-5.7')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g3], E[g3], label='btw 5.7-7.2 ')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g4], E[g4], label='above 7.2')
km_3.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km11.png',dpi=300)

In [ ]:
km_3 = KaplanMeierFitter()

groups=df['anxiety']
g1=(groups<=4.8)  
g2=(groups>4.8) & (groups<=5.6)
g3=(groups>5.6) & (groups<=7.1)
g4=(groups>7.1)
plt.figure(figsize=(5,3))
ax=plt.subplot()

## fit the model for 1st group
km_3.fit(T[g1], E[g1], label='below 4.8')
km_3.plot(ax=ax,ci_show=False)

## fit the model for 2nd group
km_3.fit(T[g2], E[g2], label='btw 4.8-5.6')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g3], E[g3], label='btw 5.6-7.1 ')
km_3.plot(ax=ax,ci_show=False)

km_3.fit(T[g4], E[g4], label='above 7.1')
km_3.plot(ax=ax,ci_show=False)


plt.grid()
plt.show()
plt.savefig('km12.png',dpi=300)

In [ ]:
# Create Model
cph = CoxPHFitter(penalizer=0.1)

# Fit the data to train the model
cph.fit(df, 'stag', event_col='event')

# Have a look at the significance of the features
cph.print_summary()


In [ ]:
cph.plot()

In [ ]:
def map_all(frame):
    # Map age
    #age varies from 18 to 58, therefore 4 categories map to range of 10.
    frame.loc[frame.age <= 26 , 'age'] = 0
    frame.loc[(frame.age >26) & (frame.age<=30) , 'age'] = 1
    frame.loc[(frame.age >30) & (frame.age<=36) , 'age'] = 2
    frame.loc[(frame.age >36) , 'age'] = 3
    
    
    # Map independ
    # independ varies from 1 to 10 and we will map it depending upon the quartile variation.
    # i.e 25% -> 4.1 , 50% -> 5.5 , 75% -> 6.9
    frame.loc[(frame.independ <= 4.1) , 'independ'] = 0
    frame.loc[(frame.independ > 4.1) & (frame.independ <= 5.5) , 'independ'] = 1
    frame.loc[(frame.independ > 5.5) & (frame.independ <= 6.9) , 'independ'] = 2
    frame.loc[(frame.independ > 6.9) , 'independ'] = 3

     # Map self control
    # selfcontrol varies from 1 to 10 and we will map it depending upon the quartile variation.
    # i.e 25% -> 4.1 , 50% -> 5.7 , 75% -> 7.2
    frame.loc[(frame.selfcontrol <= 4.1) , 'selfcontrol'] = 0
    frame.loc[(frame.selfcontrol > 4.1) & (frame.selfcontrol <= 5.7) , 'selfcontrol'] = 1
    frame.loc[(frame.selfcontrol > 5.7) & (frame.selfcontrol <= 7.2) , 'selfcontrol'] = 2
    frame.loc[(frame.selfcontrol > 7.2) , 'selfcontrol'] = 3

    # Map anxiety
    # anxiety varies from 1 to 10 and we will map it depending upon the quartile variation.
    # i.e 25% -> 4.8 , 50% -> 5.6 , 75% -> 7.1
    frame.loc[(frame.anxiety <= 4.8) , 'anxiety'] = 0
    frame.loc[(frame.anxiety > 4.8) & (frame.anxiety <= 5.6) , 'anxiety'] = 1
    frame.loc[(frame.anxiety > 5.6) & (frame.anxiety <= 7.1) , 'anxiety'] = 2
    frame.loc[(frame.anxiety > 7.1) , 'anxiety'] = 3
    

    # Map stag
    # stag varies from 0.394250513 to 179.449692 and we will map it depending upon the quartile variation.
    # i.e 25% ->11.72895277  , 50% ->24.3449692 , 75% ->51.31827515 
    frame.loc[(frame.stag <=11.72895277 ) , 'stag'] = 0
    frame.loc[(frame.stag >11.72895277 ) & (frame.stag <24.3449692) , 'stag'] = 1
    frame.loc[(frame.stag>24.3449692 ) & (frame.stag <=51.31827515 ) , 'stag'] = 2
    frame.loc[(frame.stag >51.31827515 ) , 'stag'] = 3


    
    frame.loc[(frame.Year ==2012 ) , 'Year'] = 0
    frame.loc[(frame.Year ==2013 ) , 'Year'] = 1
    frame.loc[(frame.Year ==2014 ) , 'Year'] = 2
    
    
    frame.loc[(frame.Month <=3 ) , 'Month'] = 0
    frame.loc[(frame.Month >3 ) & (frame.Month <= 6) , 'Month'] = 1
    frame.loc[(frame.Month>6 ) & (frame.Month <= 10) , 'Month'] = 2
    frame.loc[(frame.Month >10 ) , 'Month'] = 3
    
    
    frame.loc[(frame.Date <=8 ) , 'Date'] = 0
    frame.loc[(frame.Date >8 ) & (frame.Date <=16) , 'Date'] = 1
    frame.loc[(frame.Date >16 ) & (frame.Date <=23 ) , 'Date'] = 2
    frame.loc[(frame.Date >23 ) , 'Date'] = 3

In [ ]:
df.dtypes

In [ ]:
X=df.drop(['stag'],axis=1)##independent features
y=df['stag']##dependent features

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=66)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
df.dtypes

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_regressor=LinearRegression()
mse=cross_val_score(lin_regressor,X,y,scoring='neg_mean_squared_error',cv=5)
mean_mse=np.mean(mse)
print(mean_mse)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X,y)
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso=Lasso()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)

lasso_regressor.fit(X,y)
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

In [ ]:
prediction_lasso=lasso_regressor.predict(x_test)
prediction_ridge=ridge_regressor.predict(x_test)

In [ ]:
y_test

In [ ]:
import seaborn as sns

sns.distplot(y_test-prediction_lasso)

In [ ]:
import seaborn as sns

sns.distplot(y_test-prediction_ridge)